In [1]:
import math
import numpy as np
import heapq as hq
import pandas as pd

# The following function is a useful helper.

def second_argmax(x):
    ### x: numeric list (or array) of nonzero length
    ### Returns: indices of second highest and maximum elements, 
    ### where index of second highest is -1 if array has length 1.
    second_val = -np.inf
    second_argmax = -1
    max_val = x[0]
    argmax = 0
    for i in range(1,len(x)):
        if (x[i] > max_val):
            second_argmax = argmax
            second_val = max_val
            argmax = i 
            max_val = x[i]
        elif (x[i] > second_val):
            second_argmax = i
            second_val = x[i]
    return second_argmax, argmax

def second_argmin(x):
    ### x: numeric numpy array of nonzero length
    return second_argmax(- x)

In [62]:
def huntington_hill(populations_array,num_seats):
    ### populations is a numpy array of populations
    ### num_seats is the desired number of seats to assign
    num_states = len(populations_array)
    if (num_states > num_seats):
        print("More states than seats!")
        return None
    representatives = np.ones(num_states)
    priorities = populations_array / math.sqrt(2)
    last_state = None
    last_priority = None
    second_last_state = None
    second_last_state_priority = None
    for j in range(num_states,num_seats):
        highest = np.argmax(priorities) # index of state with highest priority. It will receive this seat
        representatives[highest] +=  1    
        # Update this state's priority
        priorities[highest] = populations_array[highest]/math.sqrt(
            representatives[highest] * (representatives[highest]+1))
    # After the for loop,
    # representatives contains the number of reps assigned to each state.
    # "priorities" is an array of the priority of each state fo the next (unassigned) seat. 
    # invariant: priorities = populations_array / np.sqrt(representatives * (representatives + 1))
    
    
    ############
    # Computing the distance to instability. 
    ############
    # CLAIM (no proof): the minimal change consists of 
    # either adding or removing people to/from a single state. 
    ############

    # We will need the highest and second-highest of the current priorities
    second_high, high = second_argmax(priorities)
    
    # We also need the lowest and second-lowest of the priorities that each state had 
    # when it was last assigned a seat

    prev_norm_factors = np.sqrt(representatives*(representatives - 1)) 
    #prev_priorities = populations_array / prev_norm_factors
    prev_priorities = np.zeros(num_states)
    for i in range(num_states):
        if (representatives[i] == 1):
            prev_priorities[i] = np.inf
        else: # normalization factor is not 0
            prev_priorities[i] = populations_array[i] / prev_norm_factors[i]

    second_low, low = second_argmin (prev_priorities)
        
    ############
    # Step 1: Compute the minimal population addition which would gain some state a new seat.
    # For all but one state, this is the addition of population needed to raise their 
    # priority over min(prev_priorities)
    # Let's call this its "normalized deficiency"
    normalization_factors = np.sqrt(representatives*(representatives + 1))
    normalized_defs = (prev_priorities[low] - priorities) * normalization_factors
    # Need to fix computation of normalized gap for last state. 
    # In order for it to gain a seat, it's current priority would have to move above the second 
    # highest of the priorities. 
    normalized_defs[low] = (prev_priorities[second_low] - 
                                       priorities[low]) * normalization_factors[low]

    min_addition = np.min(normalized_defs)
    min_addition_index = np.argmin(normalized_defs)

    ############
    # Step 2: Compute the number of people we would have to drop from a state to cost it a seat.
    # For all but one state, this is the number of people we need to drop to bring 
    # the state's priority at the time its last seat was assigned down to the priority 
    # of the state which would get the next seat, if there were one. 
    # We'll call this a normalized surplus

    # The natural way to write this would be the following:    
    #    normalized_surplus = (prev_priorities - priorities[high]) * prev_norm_factors 
    # but Python doesn't like arrays with np.inf values; instead, we write:
    normalized_surplus = populations_array - ( priorities[high] * prev_norm_factors )

    # Now we need to fix the surplus for the state which would have gotten the next seat. 
    normalized_surplus[high] = populations_array[high] - priorities[second_high] * prev_norm_factors[high]

    
    min_removal = np.min(normalized_surplus)
    min_removal_index = np.argmin(normalized_surplus)
        
    distance_to_instability = min(min_addition, min_removal)    
        
    stats = dict([
        ('representatives', representatives),
        ('distance to instability', distance_to_instability),
        ('priorities', priorities),
        ('previous priorities', prev_priorities),
        ('min addition', min_addition),
        ('min add inded', min_addition_index),
        ('min removal', min_removal),
        ('min rem index', min_removal_index),
        ('defs', normalized_defs),
        ('surps', normalized_surplus)
        ])
    
    return stats

## Experiments with Indian example Data

In [63]:
example_populations_table = pd.DataFrame(
    [["JAMMU & KASHMIR",10143700],["HIMACHAL PRADESH",6077900],["PUNJAB",24358999],
    ["CHANDIGARH",900635],["UTTARANCHAL",8489349],["HARYANA",21144564],["DELHI",13850507],
    ["RAJASTHAN",56507188],["UTTAR PRADESH",166197921],["BIHAR",82998509],
    ["SIKKIM",540851],["ARUNACHAL PRADESH",1097968],["NAGALAND",1990036],
    ["MANIPUR",2166788],["MIZORAM",888573],["TRIPURA",3199203],
    ["MEGHALAYA",2318822],["ASSAM",26655528],["WEST BENGAL",80176197],
    ["JHARKHAND",26945829],["ORISSA",36804660],["CHHATTISGARH",20833803],
    ["MADHYA PRADESH",60348023],["GUJARAT",50671017],["DAMAN & DIU",158204],
    ["DADRA & NAGAR HAVELI",220490],["MAHARASHTRA",96878627],["ANDHRA PRADESH",76210007],
    ["KARNATAKA",52850562],["GOA",1347668],["LAKSHADWEEP",60650],
    ["KERALA",31841374],["TAMIL NADU",62405679],["PONDICHERRY",974345],
    ["ANDAMAN & NICOBAR ISLANDS",356152]
    ] )

example_populations_array = example_populations_table[1].values

number_of_seats = 545

stats = huntington_hill(example_populations_array, number_of_seats)

for x in stats:
    print("====")
    print(x, ":")
    print(stats[x])

====
representatives :
[ 5.  3. 13.  1.  4. 11.  7. 30. 87. 43.  1.  1.  1.  1.  1.  2.  1. 14.
 42. 14. 19. 11. 32. 26.  1.  1. 51. 40. 28.  1.  1. 17. 33.  1.  1.]
====
distance to instability :
60650.0
====
priorities :
[1851977.76885505 1754538.60055381 1805609.13632984  636845.11587395
 1898276.14487199 1840398.06866262 1850851.85096836 1852943.68989505
 1899435.82287884 1908137.75484105  382439.40971153  776380.61832583
 1407167.95040535 1532150.48819364  628315.99387927 1306069.15559686
 1639654.76056455 1839406.18388858 1886629.32004193 1859438.85608284
 1888039.22079272 1813349.79544139 1857082.03218127 1912454.29486168
  111867.12121084  155909.97418382 1881227.0522851  1881871.98686931
 1854691.13109657  952945.18158811   42886.02627896 1820250.19666449
 1863063.61543597  688965.95671521  251837.49433315]
====
previous priorities :
[2268200.27433646 2481292.28461031 1950280.76920498              inf
 2450663.96519734 2016055.07399091 2137179.6287734  1915773.09375199
 192139

## How likely is noise addition to changed apportioned seats? 

In [85]:
def huntington_hill_noise(populations_array, num_seats, epsilon, num_reps):
    ### populations_array is a numpy array  containing the states' populations
    ### num_seats is the desired number of seats to assign
    ### epsilon is the parameter used for DP noise addition (Laplace noise)
    ### num_reps is the number of attempts that are made
    ### Returns a numpy array
    ref_stats = huntington_hill(populations_array, num_seats)
    ref_reps = ref_stats['representatives']
    distance_to_instab = ref_stats['distance to instability']
    changed_outputs = 0
    total_changes = 0
    for i in range(num_reps):
        noisy_pops = Laplace_Histogram(populations_array, epsilon)
        reps = huntington_hill(noisy_pops, num_seats)['representatives']
        if not np.array_equal(reps, ref_reps):
            changed_outputs = changed_outputs + 1
        total_changes = total_changes + np.linalg.norm(reps - ref_reps, ord = 1)
    if (changed_outputs == 0):
        avg_change = 0.0
    else:
        avg_change = total_changes / changed_outputs
    return changed_outputs, avg_change, distance_to_instab

def Laplace_Histogram(populations_array, epsilon):
    num_states = len(populations_array)
    return populations_array + np.random.laplace(scale = 1/epsilon, size = (num_states))

Trying this with the Indian population data:

In [86]:
(n, e, d) = huntington_hill_noise(example_populations_array, 
                      number_of_seats,
                      epsilon = 0.00003, 
                      num_reps = 100)
print("Out of 100 trials, there were {} trials where the \
apportionment totals differed from the exact totals. \
Of those, the average number of seats that changed \
was {}. The distance to instability was {}.".format(n, e, d))

Out of 100 trials, there were 18 trials where the apportionment totals differed from the exact totals. Of those, the average number of seats that changed was 2.0. The distance to instability was 60650.0.


# Experiments with US historical data

First, we load the population data into memory. 

This data comes from two sources: 
* 1790 through 1990: https://www.census.gov/population/www/censusdata/pop1790-1990.html
* 2000, 2010, and 2017 (estimated): https://en.wikipedia.org/wiki/List_of_U.S._states_and_territories_by_population 
* Note that the 2017 numbers don't actually come from a decennial census. They are just there for fun.

The data on the number of seats historically apportioned comes from 
https://en.wikipedia.org/wiki/United_States_congressional_apportionment

In [87]:
us_pops = pd.read_csv('historical_populations.csv', header = 0)

us_seats = pd.read_csv('historical_seats_apportioned.csv', header = 0)

Now let's delete the total US population as well as the District of Columbia


In [88]:
us_pops = us_pops.drop([0,9]) 

print(us_pops.columns)
us_pops.head()

Index(['Name', '2017', '2010', '2000', '1990', '1980', '1970', '1960', '1950',
       '1940', '1930', '1920', '1910', '1900', '1890', '1880', '1870', '1860',
       '1850', '1840', '1830', '1820', '1810', '1800', '1790',
       'Year of first census', 'No significant change since', 'FIPS Code'],
      dtype='object')


,Name,2017,2010,2000,1990,1980,1970,1960,1950,1940,...,1850,1840,1830,1820,1810,1800,1790,Year of first census,No significant change since,FIPS Code
1,Alabama,4833722,4779736,4447100,4040587,3893888,3444165,3266740,3061743,2832961,...,771623.0,590756.0,309527.0,127901.0,9046.0,1250.0,NaN,1800,1820,1.0
2,Alaska,739795,710231,626932,550043,401851,300382,226167,128643,72524,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1880,1880,2.0
3,Arizona,7016270,6392017,5130632,3665228,2718215,1770900,1302161,749587,499261,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1860,1870,4.0
4,Arkansas,2959373,2915918,2673400,2350725,2286435,1923295,1786272,1909511,1949387,...,209897.0,97574.0,30388.0,14273.0,1062.0,NaN,NaN,1810,1830,5.0
5,California,39536653,37253956,33871648,29760021,23667902,19953134,15717204,10586223,6907387,...,92597.0,NaN,NaN,NaN,NaN,NaN,NaN,1850,1860,6.0


We also need to process the number of seats apportioned in each Census year.

* According to Wikipeda, "Congress failed to pass any reapportionment to implement the 1920 United States Census so despite population shift, distribution of seats from 1913 remained in effect until 1933." For experiments, we will use the total from 1910. 

* For experiments with the 2017 estimates, we use the total from 2010.

Both of these fictitious numbers happen to be 435. 

In [89]:
us_seats.head()

,Year,1790,1800,1810,1820,1830,1840,1850,1860,1870,...,1910,1930,1940,1950,1960,1970,1980,1990,2000,2010
0,Number of seats apportioned,105,142,182,213,240,223,234,241,292,...,435,435,435,435,435,435,435,435,435,435


In [90]:
us_seats['1920'] = us_seats['1910']
us_seats['2017'] = us_seats['2010']

In [91]:
representatives2010 = huntington_hill(us_pops['2010'].values, num_seats = 435)['representatives']
print(representatives2010)

[ 7.  1.  9.  4. 53.  7.  5.  1. 27. 14.  2.  2. 18.  9.  4.  4.  6.  6.
  2.  8.  9. 14.  8.  4.  8.  1.  3.  4.  2. 12.  3. 27. 13.  1. 16.  5.
  5. 18.  2.  7.  1.  9. 36.  4.  1. 11. 10.  3.  8.  1.]


In [92]:
list_of_years = us_pops.columns.drop(['Name', 'Year of first census',
       'No significant change since', 'FIPS Code'])

The following code repeatedly adds Laplace noise to state populations and checks if the apportionment changes.  

In [96]:
for epsilon in [1.0, 0.1, 0.01, 0.001, 0.0001]:
    print("==============")
    print("Now trying epsilon = ", epsilon)
    print("Each row lists: year, number of seats, number of trials (out of 100) that yielded different apportionment, the average number of seats that changed, and the distance to instability for taht year's real numbers.")
    for year in list_of_years[0:13]: 
        (n,e, d) = huntington_hill_noise(us_pops[year].values, 
                          num_seats = us_seats[year][0],
                          epsilon = epsilon, 
                          num_reps = 100)
        print('{} {} {:3d}   {:.2f} {:.2f}'.format(year, us_seats[year][0], n, e, d))

Now trying epsilon =  1.0
Each row lists: year, number of seats, number of trials (out of 100) that yielded different apportionment, the average number of seats that changed, and the distance to instability for taht year's real numbers.
2017 435   0   0.00 7786.55
2010 435   0   0.00 12408.13
2000 435   0   0.00 691.25
1990 435   0   0.00 835.49
1980 435   0   0.00 7421.60
1970 435   0   0.00 692.97
1960 435   0   0.00 11435.18
1950 435   0   0.00 1875.56
1940 435   0   0.00 1179.56
1930 435   0   0.00 117.07
1920 435   0   0.00 3938.67
1910 435   0   0.00 5158.55
1900 386   0   0.00 4914.92
Now trying epsilon =  0.1
Each row lists: year, number of seats, number of trials (out of 100) that yielded different apportionment, the average number of seats that changed, and the distance to instability for taht year's real numbers.
2017 435   0   0.00 7786.55
2010 435   0   0.00 12408.13
2000 435   0   0.00 691.25
1990 435   0   0.00 835.49
1980 435   0   0.00 7421.60
1970 435   0   0.00 692.9

In [ ]:
## Need to write code to handle years with missing populations. 